# Práctica 5: Reconocimiento de Formas: Evaluación de rendimiento

* **Alumno 1**: Víctor Nieves Sánchez
* **Alumno 2**: Javier Barragán Haro


## Clasificador de la distancia euclídea

In [8]:
import numpy as np
import pandas as pd
from sklearn.covariance import ShrunkCovariance
from sklearn.base import BaseEstimator
from sklearn import preprocessing
from abc import abstractmethod


class Classifier(BaseEstimator):

    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass

class ClassifEuclid(Classifier):
    def __init__(self):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        self.centroids = None
        self.distancias = None

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        self.distancias = np.array(np.unique(y))
        # Entrena el clasificador
        self.centroids = np.array([np.mean(X[y==i], axis=0) for i in np.unique(y)])
        return self

    def decision_function(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas.
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        assert self.centroids is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.centroids.shape[1]
        return np.linalg.norm(self.centroids - X[:, np.newaxis], axis=2)

    def predict(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        # Calcula y devuelve la clase más probable
        return self.distancias[np.argmin(self.decision_function(X), axis=1)]

    def  num_aciertos(self,X,y):
        estimation = self.predict(X)
        aciertos = (estimation == y).sum()
        return aciertos, aciertos/len(y) * 100

## Clasificador Bayesiano Paramétrico

In [11]:
class ClassifBayesianoParametrico(Classifier):
    def __init__(self, share_covs=False, shrinkage=0.0):
        """Constructor de la clase
        labels: lista de etiquetas de esta clase"""
        assert 0 <= shrinkage <= 1
        self.labels = None
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
        self.share_covs = share_covs
        self.shrinkage = shrinkage
        self.scaler = preprocessing.StandardScaler()

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        assert np.max(y) < len(np.unique(y)), "Error: Las etiquetas deben ser enteros entre 0 y el número de clases.\n" \
                                              "Puedes hacer esto empleando:\ny = pd.factorize(y)[0]"
        # Entrena el clasificador

        # Preprocesamos los datos de entrada
        X = self.scaler.fit_transform(X)       

        # Contar cuantos ejemplos hay de cada etiqueta
        self.labels, counts = np.unique(y, return_counts=True)

        # Usando el contador de ejemplos de cada etiqueta, calcular el logaritmo neperiano de las probabilidades a-priori
        self.ln_apriories = np.array([np.log((counts[i]/np.sum(counts))) for i in self.labels])

        # Calcular para los ejemplos de cada clase, la media de cada una de sus características (centroide)
        self.means = np.array([np.mean(X[y==i], axis=0) for i in self.labels])
        
        if self.share_covs:
            # Restamos al dato de cada clase su centroide
            X_mean = np.array([])
            for i in self.labels:
                X_mean = np.append(X_mean.reshape(-1, X.shape[1]), X[y==i] - self.means[i], axis=0)

            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            cov = ShrunkCovariance(shrinkage=self.shrinkage).fit(X_mean).covariance_

            # La reproducimos tantas veces como número de clases
            covs = np.tile(cov, (len(self.labels), 1, 1))
         
        else:
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            covs = np.array([ShrunkCovariance(shrinkage=self.shrinkage).fit(X[y==i]).covariance_ for i in self.labels])

        # Para cada una de las clases, calcular el logaritmo neperiano de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.ln_determinants = np.log(list(map(lambda x: np.linalg.det(x), covs)))

        # Para cada una de las clases, calcular la inversa de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.inv_covs = np.array(list(map(lambda x: np.linalg.pinv(x), covs)))

        return self

    def predict_proba(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas.
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]
        
        # Calcula y devuelve la probabilidad de cada clase
        
        # Preprocesamos nuestros datos
        X = self.scaler.fit_transform(X)        

        # Resta la media de cada clase a cada ejemplo en X  
        X_mean0 = np.array(X[:, np.newaxis] - self.means)

        # Calcula el logaritmo de la función de decisión gausiana
        # -(1/2)ln|Sigma_i| - (1/2)*(x- mu_i)^T Sigma_i^-1 (x- mu_i) + lnP(alpha_i)
        grado_de_pertenencia = np.array(list(map(lambda x: -(1/2)*(self.ln_determinants 
                   + np.matmul(np.matmul(x.reshape(len(self.labels),1,X.shape[1]),self.inv_covs), 
                   x.reshape(len(self.labels),1,X.shape[1]).transpose(0,2,1)).reshape(len(self.labels)))
                   + self.ln_apriories, X_mean0)))

        return grado_de_pertenencia

    def predict(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        # Calcula y devuelve la clase más probable
        return self.labels[np.argmax(self.predict_proba(X), axis=1)]

    def  num_aciertos(self,X,y):
        estimation = self.predict(X)
        aciertos = (estimation == y).sum()
        return aciertos, aciertos/len(y) * 100

In [18]:
def best_shrinkage_clf(X, y, k, shrinkages, share_covs):
    """
    Busca el clasificador bayesiano regularizado con el mejor shrinkage. 
    :param X: Ejemplos de la dase de datos
    :param y: Etiquetas de los ejemplos
    :param k: Número de divisiones en la validación cruzada (k-fold)
    :param shrinkages: Lista de posibles shrinkages que conforman la rejilla de búsqueda
    :param share_covs: Lista de posibles valores para share_covs que conforman la rejilla de búsqueda
    """
    from sklearn.model_selection import GridSearchCV
    cbp = ClassifBayesianoParametrico()
    params = {'shrinkage': shrinkages, 'share_covs': share_covs}
    clf = GridSearchCV(cbp, params, n_jobs=-2, scoring='accuracy', cv=k).fit(X, y)
    best_clf = clf.best_estimator_
    # print("Srinkage scores: ", clf.cv_results_['mean_test_score'])
    result_score_mean = clf.cv_results_['mean_test_score'][clf.best_index_]
    result_score_std = clf.cv_results_['std_test_score'][clf.best_index_]
    print("\tSelected shrinkage = {}, share_covs = {}\n" \
          "\tAccuracy: {:.3f} (+/- {:.3f})\n".format(best_clf.shrinkage,
                                                   best_clf.share_covs,
                                                   result_score_mean,
                                                   result_score_std))
    return best_clf

## Iris Dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Iris:

In [20]:
import sklearn
from time import time
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
from sklearn.svm import SVC

# 1. Cargar los datos de la base de datos de entrenamiento
dataset = load_iris()
X = dataset.data
y = pd.factorize(dataset.target)[0]


# 2. Baraja los datos para hacer validación cruzada
X, y = sklearn.utils.shuffle(X, y)

# Evaluar el clasificador de la distancia eucídea usando cross validation (k-fold=5)
k = 5
print("\033[1m", "Clasificador Euclídeo", "\033[0m", "\n")
cde = SVC(kernel='linear', C=1).fit(X,y)
scoresCde = cross_val_score(cde, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresCde.mean(), scoresCde.std() * 2), "\n")
mConfusionCde = confusion_matrix(y, cde.predict(X) , labels=np.unique(y))
print("Matriz de confusión con clasificador de la distancia euclidea: \n", mConfusionCde, "\n")

# Haz una selección de modelos para buscar shrinkage del clasificador 
# estadístico paramétrico que obtiene el mejor accuracy. Usa para ello cross validation (k-fold=5)
print("\033[1m", "Clasificador Estadístico Paramétrico", "\033[0m", "\n")

clf = best_shrinkage_clf(X, y, k, [1.0, 0.8, 0.6, 0.5, 0.4, 0.2, 0.1, 0.0], [True, False])
cbp = ClassifBayesianoParametrico(clf.share_covs, clf.shrinkage).fit(X,y)
scoresCbp = cross_val_score(cbp, X, y, scoring='accuracy', cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresCbp.mean(), scoresCbp.std() * 2), "\n")

mConfusionCbp = confusion_matrix(y, cbp.predict(X) , labels=np.unique(y))
print("Matriz de confusión con clasificador bayesiano parametrico: \n", mConfusionCbp, "\n")

 Clasificador Euclídeo  

Accuracy: 0.99 (+/- 0.03) 

Matriz de confusión con clasificador de la distancia euclidea: 
 [[50  0  0]
 [ 0 49  1]
 [ 0  0 50]] 

 Clasificador Estadístico Paramétrico  

	Selected shrinkage = 0.1, share_covs = True
	Accuracy: 0.953 (+/- 0.062)

Accuracy: 0.95 (+/- 0.12) 

Matriz de confusión con clasificador bayesiano parametrico: 
 [[50  0  0]
 [ 0 47  3]
 [ 0  1 49]] 



## Wine dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Wine:

In [21]:
import sklearn
from time import time
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_wine
dataset = load_wine()
X = dataset.data
y = pd.factorize(dataset.target)[0]

# 2. Baraja los datos para hacer validación cruzada
X, y = sklearn.utils.shuffle(X, y)

# Evaluar el clasificador de la distancia eucídea usando cross validation (k-fold=5)
k = 5
print("\033[1m", "Clasificador Euclídeo", "\033[0m", "\n")
cde = SVC(kernel='linear', C=1).fit(X,y)
scoresCde = cross_val_score(cde, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresCde.mean(), scoresCde.std() * 2), "\n")
mConfusionCde = confusion_matrix(y, cde.predict(X) , labels=np.unique(y))
print("Matriz de confusión con clasificador de la distancia euclidea: \n", mConfusionCde, "\n")

# Haz una selección de modelos para buscar shrinkage del clasificador 
# estadístico paramétrico que obtiene el mejor accuracy. Usa para ello cross validation (k-fold=5)
print("\033[1m", "Clasificador Estadístico Paramétrico", "\033[0m", "\n")

clf = best_shrinkage_clf(X, y, k, [1.0, 0.8, 0.6, 0.5, 0.4, 0.2, 0.1, 0.0], [True, False])
cbp = ClassifBayesianoParametrico(clf.share_covs, clf.shrinkage).fit(X,y)
scoresCbp = cross_val_score(cbp, X, y, scoring='accuracy', cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresCbp.mean(), scoresCbp.std() * 2), "\n")
mConfusionCbp = confusion_matrix(y, cbp.predict(X) , labels=np.unique(y))
print("Matriz de confusión con clasificador bayesiano parametrico: \n", mConfusionCbp, "\n")

 Clasificador Euclídeo  

Accuracy: 0.97 (+/- 0.05) 

Matriz de confusión con clasificador de la distancia euclidea: 
 [[59  0  0]
 [ 0 70  1]
 [ 0  0 48]] 

 Clasificador Estadístico Paramétrico  

	Selected shrinkage = 0.4, share_covs = False
	Accuracy: 0.994 (+/- 0.011)

Accuracy: 0.99 (+/- 0.02) 

Matriz de confusión con clasificador bayesiano parametrico: 
 [[59  0  0]
 [ 0 70  1]
 [ 0  0 48]] 



## Breast cancer dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Breast cancer:

In [22]:
import sklearn
from time import time
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
X = dataset.data
y = pd.factorize(dataset.target)[0]

# 2. Baraja los datos para hacer validación cruzada
X, y = sklearn.utils.shuffle(X, y)

# Evaluar el clasificador de la distancia eucídea usando cross validation (k-fold=5)
k = 5
print("\033[1m", "Clasificador Euclídeo", "\033[0m", "\n")
cde = SVC(kernel='linear', C=1).fit(X,y)
scoresCde = cross_val_score(cde, X, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresCde.mean(), scoresCde.std() * 2), "\n")
mConfusionCde = confusion_matrix(y, cde.predict(X) , labels=np.unique(y))
print("Matriz de confusión con clasificador de la distancia euclidea: \n", mConfusionCde, "\n")

# Haz una selección de modelos para buscar shrinkage del clasificador 
# estadístico paramétrico que obtiene el mejor accuracy. Usa para ello cross validation (k-fold=5)
print("\033[1m", "Clasificador Estadístico Paramétrico", "\033[0m", "\n")

clf = best_shrinkage_clf(X, y, k, [1.0, 0.8, 0.6, 0.5, 0.4, 0.2, 0.1, 0.0], [True, False])
cbp = ClassifBayesianoParametrico(clf.share_covs, clf.shrinkage).fit(X,y)
scoresCbp = cross_val_score(cbp, X, y, scoring='accuracy', cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scoresCbp.mean(), scoresCbp.std() * 2), "\n")
mConfusionCbp = confusion_matrix(y, cbp.predict(X) , labels=np.unique(y))
print("Matriz de confusión con clasificador bayesiano parametrico: \n", mConfusionCbp, "\n")

 Clasificador Euclídeo  

Accuracy: 0.95 (+/- 0.01) 

Matriz de confusión con clasificador de la distancia euclidea: 
 [[201  11]
 [  8 349]] 

 Clasificador Estadístico Paramétrico  

	Selected shrinkage = 0.2, share_covs = True
	Accuracy: 0.958 (+/- 0.017)

Accuracy: 0.96 (+/- 0.03) 

Matriz de confusión con clasificador bayesiano parametrico: 
 [[191  21]
 [  1 356]] 



In [23]:
class ExclusionSplitter:
    """Esta clase nos permite usar GridSearchCV con la evaluación por exclusion."""
    def __init__(self, train_indices, test_indices):
        self.train_indices = train_indices
        self.test_indices = test_indices

    def split(self, X, y=None, groups=None):
        return [(self.train_indices, self.test_indices)]

    def get_n_splits(self, X=None, y=None, groups=None):
        return 1

## Isolet Dataset (Isolated Letter Speech Recognition)
Carga, entrenamiento, predicción y evaluación en la base de datos de Isolet

In [28]:
import pandas as pd
import numpy as np
import os
from sklearn.datasets import fetch_openml

# Loading MNIST from Drive
from google.colab import drive
drive.mount('/content/gdrive')
isolet_dir = '/content/gdrive/My Drive/Colab Notebooks/isolet_subsets'

# isolet_dir = 'isolet_subsets'

train_indices = np.load(os.path.join(isolet_dir, 'isolet_train.npy'))
test_indices = np.load(os.path.join(isolet_dir, 'isolet_test.npy'))

# 1. Cargar los datos de la base de datos de entrenamiento
X_all, y_all = fetch_openml('isolet', version=1, return_X_y=True, cache=True)
y_all = pd.factorize(y_all)[0]
X_train, y_train = X_all[train_indices], y_all[train_indices]
X_test, y_test = X_all[test_indices], y_all[test_indices]

ModuleNotFoundError: No module named 'google.colab'

In [29]:
import numpy as np

# Dado el tamaño de la BD vamos emplear Exclusión como método de evaluación

print("\033[1m", "Clasificador Euclídeo", "\033[0m", "\n")
# 2. Entrena el clasificador de la distancia eucídea empleando X_train e y_train
cde = ClassifEuclid()
cde.fit(X_train,y_train)

# 3. Evalua el rendimiento en la base de datos de entrenamiento (X_train, y_train)
scoresCde = cde.num_aciertos(X_train, y_train)
print("Entrenamiento C. distancia euclideo: ", scoresCde, "\n")

# 4. Evalua el rendimiento en la base de datos de test (X_test, y_test)
scoresCde = cde.num_aciertos(X_test, y_test)
print("Test C. distancia euclideo: ", scoresCde, "\n")

print("\033[1m", "Clasificador Estadístico Paramétrico", "\033[0m", "\n")
from sklearn.model_selection import GridSearchCV
# Creamos un objeto spliter para dividir la base de datos en Train y test
splitter = ExclusionSplitter(np.arange(0, len(X_train)), 
                             np.arange(len(X_train), len(X_train) + len(X_test)))

# Decidimos los parámetros que vamos a pasar a GridSearchCV para hacer la búsqueda en rejilla
# NOTA: Ten cuidado! Un gran número de celdas en la rejilla ralentizará la ejecución
# Ejemplo de uso: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
params = {'shrinkage': [1.0, 0.5, 0.0], 'share_covs': [True]}

# Definimos la búsqueda en rejilla. 
clf = GridSearchCV(ClassifBayesianoParametrico(), params, scoring='accuracy', cv=splitter, verbose=1)
# Ejecutamos la búsqueda
clf = clf.fit(X_all, y_all)
# Seleccionamos el clasificador que mojores resultados ha obtenido 
best_clf = clf.best_estimator_
# Mostramos los resultados
print("Srinkage scores: ", clf.cv_results_['mean_test_score'])
result_score_mean = clf.cv_results_['mean_test_score'][clf.best_index_]
result_score_std = clf.cv_results_['std_test_score'][clf.best_index_]
print("\tSelected shrinkage = {} Share covs = {} \n\tAccuracy: {:.3f} (+/- {:.3f})".format(best_clf.shrinkage,
                                                                                           best_clf.share_covs,
                                                                                           result_score_mean,
                                                                                           result_score_std))

 Clasificador Euclídeo  



NameError: name 'X_train' is not defined

## MNIST Database (Modified National Institute of Standards and Technology database)
MNIST es una base de datos de texto manuscrito, que se usa de forma clásica para entrenar sistemas de procesado de imágenes.

Carga, entrenamiento, predicción y evaluación en la base de datos de MNIST:

In [32]:
import idx2numpy
import os
import numpy as np

def load_mnsit_data(mnist_dir='MNIST', training_split=True):
    """
    This function load the MNIST dataset from a disk directory.
    :param mnist_dir: The directory where the MNIST Dataset is stored. The directory should contain the sub-directories:
        - t10k-images-idx3-ubyte
        - t10k-labels-idx1-ubyte
        - train-images-idx3-ubyte
        - train-labels-idx1-ubyte
    :param training_split: True if we want to load the training examples and data, False for testing.
    :return: A tuple with the training examples X in the first element and the y labels in the second one.
    """
    assert type(mnist_dir) == str and type(training_split) == bool
    assert os.path.isdir(mnist_dir), "Error: The directory \"{}\" does not exists.".format(mnist_dir)
    file = os.path.join(mnist_dir, 'train-images-idx3-ubyte' if training_split else 't10k-images-idx3-ubyte', 'data')
    X = idx2numpy.convert_from_file(file)
    X = X.reshape(X.shape[0], -1).astype(float)
    file = os.path.join(mnist_dir, 'train-labels-idx1-ubyte' if training_split else 't10k-labels-idx1-ubyte', 'data')
    y = idx2numpy.convert_from_file(file)
    return X, y


# Loading MNIST from Drive
from google.colab import drive

drive.mount('/content/gdrive')
drive_dir = '/content/gdrive/My Drive/Colab Notebooks/MNIST'
X_train, y_train = load_mnsit_data(drive_dir, training_split=True)
X_test, y_test = load_mnsit_data(drive_dir, training_split=False)

# X_train, y_train = load_mnsit_data(training_split=True)
# X_test, y_test = load_mnsit_data(training_split=False)

X_all = np.append(X_train, X_test, axis=0)
y_all = np.append(y_train, y_test, axis=0)

ModuleNotFoundError: No module named 'google.colab'

In [33]:
import numpy as np

# Dado el tamaño de la BD vamos emplear Exclusión como método de evaluación

print("\033[1m", "Clasificador Euclídeo", "\033[0m", "\n")
# 2. Entrena el clasificador de la distancia eucídea empleando X_train e y_train
cde = ClassifEuclid()
cde.fit(X_train,y_train)

# 3. Evalua el rendimiento en la base de datos de entrenamiento (X_train, y_train)
scoresCde = cde.num_aciertos(X_train, y_train)
print("Entrenamiento C. distancia euclideo: ", scoresCde, "\n")

# 4. Evalua el rendimiento en la base de datos de test (X_test, y_test)
scoresCde = cde.num_aciertos(X_test, y_test)
print("Test C. distancia euclideo: ", scoresCde, "\n")

print("\033[1m", "Clasificador Estadístico Paramétrico", "\033[0m", "\n")
from sklearn.model_selection import GridSearchCV

# Creamos un objeto spliter para dividir la base de datos en Train y test
splitter = ExclusionSplitter(np.arange(0, len(X_train)), 
                             np.arange(len(X_train), len(X_train) + len(X_test)))

# Decidimos los parámetros que vamos a pasar a GridSearchCV para hacer la búsqueda en rejilla
# NOTA: Ten cuidado! Un gran número de celdas en la rejilla ralentizará la ejecución
# Ejemplo de uso: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
params = {'shrinkage': [1.0, 0.5, 0.0], 'share_covs': [True]}

# Definimos la búsqueda en rejilla. 
clf = GridSearchCV(ClassifBayesianoParametrico(), params, scoring='accuracy', cv=splitter, verbose=1)
# Ejecutamos la búsqueda
clf = clf.fit(X_all, y_all)
# Seleccionamos el clasificador que mojores resultados ha obtenido 
best_clf = clf.best_estimator_
# Mostramos los resultados
print("Srinkage scores: ", clf.cv_results_['mean_test_score'])
result_score_mean = clf.cv_results_['mean_test_score'][clf.best_index_]
result_score_std = clf.cv_results_['std_test_score'][clf.best_index_]
print("\tSelected shrinkage = {} Share covs = {} \n\tAccuracy: {:.3f} (+/- {:.3f})".format(best_clf.shrinkage,
                                                                                           best_clf.share_covs,
                                                                                           result_score_mean,
                                                                                           result_score_std))

 Clasificador Euclídeo  



NameError: name 'X_train' is not defined

Resultados de los experimentos. Pon el mejor de los resultados:

| Base de datos | Accuracy | Desviación Típica |
| --- | --- | --- |
| Iris   |||
| Wine   | ||
| Cancer |||
| MNIST  | ||
| Isolet |||